# 第8章: ニューラルネット

第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras

## 70. 単語ベクトルの和による特徴量

### データ準備

In [2]:
# 学習データ，検証データ，評価データ読み込み
df_train = pd.read_csv('train.txt', sep='\t')
df_valid = pd.read_csv('valid.txt', sep='\t')
df_test = pd.read_csv('test.txt', sep='\t')
df_test.head()

,CATEGORY,TITLE
0,b,Vodafone's Service Revenue Falls as European M...
1,e,"So, Tupac Shakur's Final Words Were A Big ""F*C..."
2,t,Elephants really are intelligent: Creatures ca...
3,t,FTC says Snapchat deceived customers over 'dis...
4,b,UPDATE 3-GM says facing multiple probes into r...


### ラベルベクトル作成

In [5]:
def trans_label_int(label: str) -> int:
    if label == 'b':
        return 0
    elif label == 't':
        return 1
    elif label == 'e':
        return 2
    elif label == 'm':
        return 3

print(trans_label_int(label='b'))
print(trans_label_int(label='t'))
print(trans_label_int(label='e'))
print(trans_label_int(label='m'))

0
1
2
3


In [11]:
# ラベルを整数に変換
y_train = (
    df_train
    .loc[:, 'CATEGORY']
    .apply(trans_label_int)
    .values
)
y_valid = (
    df_valid
    .loc[:, 'CATEGORY']
    .apply(trans_label_int)
    .values
)
y_test = (
    df_test
    .loc[:, 'CATEGORY']
    .apply(trans_label_int)
    .values
)

In [13]:
np.save('70_y_train.npy', y_train)
np.save('70_y_valid.npy', y_valid)
np.save('70_y_test.npy', y_test)

### 特徴量行列作成

#### 単語の出現回数取得

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# 試しに動かす
vectorizer = CountVectorizer()

tes = pd.DataFrame([
    'This is my car',
    'This is my friend',
    'This is my English book',
], columns=['TITLE'])

tes_vector = vectorizer.fit_transform(tes['TITLE'].tolist())

print(vectorizer.get_feature_names())
print(tes_vector.toarray())

['book', 'car', 'english', 'friend', 'is', 'my', 'this']
[[0 1 0 0 1 1 1]
 [0 0 0 1 1 1 1]
 [1 0 1 0 1 1 1]]


In [5]:
vectorizer = CountVectorizer()
# 計算 & 変換
train_data = vectorizer.fit_transform(df_train['TITLE'].tolist()).toarray()
feature_names = vectorizer.get_feature_names()
# 変換
valid_data = vectorizer.transform(df_valid['TITLE'].tolist()).toarray()
test_data = vectorizer.transform(df_test['TITLE'].tolist()).toarray()

In [19]:
type(feature_names)

list

In [31]:
def trans_title_word_cnt(array: np.ndarray, columns: list) -> pd.DataFrame:
    """
    入力
    array: 単語の出現回数
    columns: 単語リスト
    出力
    データフレーム
    - Title No
    - 単語
    - 出現回数
    """
    df = (
        pd.DataFrame(array, columns=columns)
        .applymap(lambda x: np.nan if x == 0 else x)
        .stack()
        .reset_index()
    )
    df.columns = ['No', 'word', 'cnt']
    return df

tes_array = np.array([[0, 1, 2], [3, 4, 5]])
tes_columns = ['a', 'b', 'c']
trans_title_word_cnt(array=tes_array, columns=tes_columns)

,No,word,cnt
0,0,b,1.0
1,0,c,2.0
2,1,a,3.0
3,1,b,4.0
4,1,c,5.0


In [32]:
# 縦持ちのDataFrame
df_train_word_count = trans_title_word_cnt(array=train_data, columns=feature_names)
df_valid_word_count = trans_title_word_cnt(array=valid_data, columns=feature_names)
df_test_word_count = trans_title_word_cnt(array=test_data, columns=feature_names)

In [33]:
df_train_word_count.head()

,No,word,cnt
0,0,abs,1.0
1,0,auto,1.0
2,0,consumer,1.0
3,0,criteria,1.0
4,0,emea,1.0


#### タイトルごとの単語ベクトル平均値算出

In [34]:
from gensim.models import KeyedVectors

In [35]:
# 学習済み単語ベクトル
vector_path = os.path.join(os.getcwd(), '../data/GoogleNews-vectors-negative300.bin')

In [36]:
# 読み込み
wv_from_bin = KeyedVectors.load_word2vec_format(vector_path, binary=True)

In [107]:
# word列の単語ベクトルを返す
def get_word_vector(row, vector) -> pd.DataFrame:
    try:
        return vector[row['word']] * row['cnt']
    except KeyError:
        ret_len = len(vector['an']) # 適当な単語で長さ取得
        return np.zeros(ret_len)

(
    pd.DataFrame(
        [
            [1, 'shakur', 1.0],
            [0, 'abs', 1.0],
            [0, 'auto', 1.0],
            [1, 'auto', 2.0],
        ],
        columns=['No', 'word', 'cnt']
    )
    .apply(get_word_vector, axis=1, result_type='expand', vector=wv_from_bin)
)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.108887,0.029907,0.373047,0.371094,-0.038818,0.283203,-0.051025,-0.141602,-0.229492,0.097168,...,0.018677,-0.166992,-0.267578,-0.345703,0.322266,-0.024048,0.074219,0.072754,-0.308594,-0.190430
2,0.166992,0.201172,0.086914,0.189453,-0.137695,0.206055,0.042236,0.081543,0.126953,-0.037598,...,-0.310547,-0.038086,0.188477,-0.100586,-0.038330,0.149414,-0.176758,0.353516,-0.000847,-0.357422
3,0.333984,0.402344,0.173828,0.378906,-0.275391,0.412109,0.084473,0.163086,0.253906,-0.075195,...,-0.621094,-0.076172,0.376953,-0.201172,-0.076660,0.298828,-0.353516,0.707031,-0.001694,-0.714844


In [108]:
def concat_df_wordvector(df: pd.DataFrame, vector) -> pd.DataFrame:
    return pd.concat(
        [
            df,
            df.apply(
                get_word_vector,
                axis=1,
                result_type='expand',
                vector=wv_from_bin
            )
        ],
        axis=1
    )

tes_df = (
    pd.DataFrame(
        [
            [1, 'shakur', 1.0],
            [0, 'abs', 1.0],
            [0, 'auto', 1.0],
            [1, 'auto', 2.0],
        ],
        columns=['No', 'word', 'cnt']
    )
)
concat_df_wordvector(df=tes_df, vector=wv_from_bin)

,No,word,cnt,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,1,shakur,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,abs,1.0,0.108887,0.029907,0.373047,0.371094,-0.038818,0.283203,-0.051025,...,0.018677,-0.166992,-0.267578,-0.345703,0.322266,-0.024048,0.074219,0.072754,-0.308594,-0.190430
2,0,auto,1.0,0.166992,0.201172,0.086914,0.189453,-0.137695,0.206055,0.042236,...,-0.310547,-0.038086,0.188477,-0.100586,-0.038330,0.149414,-0.176758,0.353516,-0.000847,-0.357422
3,1,auto,2.0,0.333984,0.402344,0.173828,0.378906,-0.275391,0.412109,0.084473,...,-0.621094,-0.076172,0.376953,-0.201172,-0.076660,0.298828,-0.353516,0.707031,-0.001694,-0.714844


In [109]:
# 縦持ちデータフレームから、
def calculate_titile_wordvector(df: pd.DataFrame, vector) -> pd.DataFrame:
    return (
        concat_df_wordvector(df=df, vector=wv_from_bin)
        .drop(['word', 'cnt'], axis=1)
        .groupby('No')
        .mean()
    )

tes_df = (
    pd.DataFrame(
        [
            [0, 'abs', 1.0],
            [0, 'auto', 1.0],
            [1, 'auto', 2.0],
            [1, 'shakur', 1.0]
        ],
        columns=['No', 'word', 'cnt']
    )
)

calculate_titile_wordvector(df=tes_df, vector=wv_from_bin)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
No,,,,,,,,,,,,,,,,,,,,,
0,0.137939,0.115540,0.229980,0.280273,-0.088257,0.244629,-0.004395,-0.030029,-0.051270,0.029785,...,-0.145935,-0.102539,-0.039551,-0.223145,0.141968,0.062683,-0.051270,0.213135,-0.154720,-0.273926
1,0.166992,0.201172,0.086914,0.189453,-0.137695,0.206055,0.042236,0.081543,0.126953,-0.037598,...,-0.310547,-0.038086,0.188477,-0.100586,-0.038330,0.149414,-0.176758,0.353516,-0.000847,-0.357422


In [111]:
# titleごとの平均単語ベクトル
df_train_features = calculate_titile_wordvector(df=df_train_word_count, vector=wv_from_bin)
df_valid_features = calculate_titile_wordvector(df=df_valid_word_count, vector=wv_from_bin)
df_test_features = calculate_titile_wordvector(df=df_test_word_count, vector=wv_from_bin)

In [115]:
np.save('70_x_train.npy', df_train_features.values)
np.save('70_x_valid.npy', df_valid_features.values)
np.save('70_x_test.npy', df_test_features.values)

## 71. 単層ニューラルネットワークによる予測

In [3]:
x_train = np.load('70_x_train.npy')
x_valid = np.load('70_x_valid.npy')
x_test = np.load('70_x_test.npy')

In [4]:
y_train = np.load('70_y_train.npy')
y_valid = np.load('70_y_valid.npy')
y_test = np.load('70_y_test.npy')

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(300,)),
  tf.keras.layers.Dense(4)
])

In [6]:
y_pred_1 = model(x_train[:1]).numpy()
y_pred_1

array([[-0.11592183, -0.02461668, -0.10518386, -0.08879694]],
      dtype=float32)

In [7]:
y_tilde_1 = tf.nn.softmax(y_pred_1).numpy()
y_tilde_1

array([[0.2419026 , 0.26502928, 0.24451414, 0.24855398]], dtype=float32)

In [8]:
Y_pred = model(x_train[:4]).numpy()
Y_pred

array([[-0.11592183, -0.02461668, -0.10518388, -0.08879694],
       [ 0.06998681, -0.01152183,  0.08780163, -0.00214703],
       [ 0.15387803,  0.1208681 ,  0.0007768 ,  0.00587393],
       [ 0.04014175, -0.15812434, -0.1054402 , -0.10943846]],
      dtype=float32)

In [9]:
Y_tilde = tf.nn.softmax(Y_pred).numpy()
Y_tilde

array([[0.2419026 , 0.26502928, 0.24451412, 0.24855398],
       [0.25839087, 0.23816526, 0.2630353 , 0.2404085 ],
       [0.27114972, 0.2623452 , 0.23265807, 0.233847  ],
       [0.28202856, 0.23130617, 0.24381906, 0.24284616]], dtype=float32)

## 72. 損失と勾配の計算

In [10]:
# クロスエントロピー損失を求める関数
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
loss_fn(y_train[:1], y_tilde_1).numpy()

1.3944324

In [13]:
loss_fn(y_train[:4], Y_tilde).numpy()

1.3814249

## 73. 確率的勾配降下法による学習